In [3]:
import mygrad as mg
import numpy as np

<문제: 주어진 조건을 만족하는 Tensor 생성하기>

구간 $[0, \pi]$에 등간격으로 분포한 15개의 구성요소로 이루어진 shape-(15,)인 tensor를 만들어보자. (Hint: mg.linspace(), mg.pi)

In [6]:
# 여기에 코드 작성
mg.linspace(0, mg.pi, 15)

Tensor([0.        , 0.22439948, 0.44879895, 0.67319843, 0.8975979 ,
        1.12199738, 1.34639685, 1.57079633, 1.7951958 , 2.01959528,
        2.24399475, 2.46839423, 2.6927937 , 2.91719318, 3.14159265])

<문제: MyGrad의 기본 수학 연산>

아래와 같이 정의된 텐서 x에 대해 여러 가지 수학 연산을 적용하여 여러 가지 텐서를 구해보자.

In [4]:
x = mg.Tensor([[ 0.,  1.,  2.,  3.],
...                [ 4.,  5.,  6.,  7.],
...                [ 8.,  9., 10., 11.],
...                [12., 13., 14., 15.]])

In [6]:
x[3][1], x[3][3]

(Tensor(13.), Tensor(15.))

1. x의 3행의 첫번째, 세번째 원소에 대한 자연로그 값으로 채워진 shape-(2,)인 Tensor를 구해보자.

In [10]:
# 여기에 코드 작성
ans = mg.Tensor([[np.log(x[3][1])], [np.log(x[3][3])]]) # 
print(ans, ans.shape)

Tensor([[2.56494936],
        [2.7080502 ]]) (2, 1)


2. x를 가로, 세로로 4등분한 각 귀퉁이(왼쪽 위, 오른쪽 위, 왼쪽 아래, 오른쪽 아래)의 4개 원소를 더하여 shape-(2,2)인 Tensor를 구해보자.

In [9]:
# 여기에 코드 작성
ans = mg.Tensor([[x[0][0], x[0][-1]], [x[-1][0], x[-1][-1]]])
print(ans, ans.shape)

Tensor([[ 0.,  3.],
        [12., 15.]]) (2, 2)


3. x의 각 열의 평균을 구하여 shape-(4,)인 Tensor를 구해보자.

In [15]:
# 여기에 코드 작성
ans = mg.Tensor(np.mean(x, axis=0))
print(ans, ans.shape)

Tensor([6., 7., 8., 9.]) (4,)


4. x의 각 행을 벡터로보고, 각 벡터가 크기가 1이 되도록 정규화하여 shape-(4,4)인 Tensor로 업데이트해보자.

In [41]:
np.sqrt(np.sum(x**2, axis=1))

Tensor([ 3.74165739, 11.22497216, 19.13112647, 27.09243437])

In [73]:
# 여기에 코드 작성
t = x.data
a = np.sqrt(np.sum(t ** 2, axis = 1))
k = [t[i] / a[i] for i in range(len(t))]
x = mg.Tensor(k)
x

Tensor([[0.        , 0.26726124, 0.53452248, 0.80178373],
        [0.35634832, 0.4454354 , 0.53452248, 0.62360956],
        [0.4181667 , 0.47043754, 0.52270837, 0.57497921],
        [0.44292808, 0.47983876, 0.51674943, 0.5536601 ]])

In [74]:
# 정규화가 잘 되었는지 확인하기
(x**2).sum(axis=1)

Tensor([1., 1., 1., 1.])

f와 F가 의존하는 모든 변수들이 MyGrad의 텐서로 저장되어 있었고, f와 F를 이루는 모든 수학적 연산이 MyGrad에서 제공하는 함수였기 때문에 MyGrad를 통해 F의 모든 편미분계수를 구할 수 있었다. 이렇게 구한 편미분계수들로부터 함수의 그래디언트를 이용하는 경사하강법을 쉽게 수행할 수 있다.

<문제: 텐서 객체의 backward() 메서드 사용>

여러가지 수식으로 정의된 x에 대한 함수 F에 대해, x=2.5에서 $\frac{d F}{d x}\big|_{x=2.5}$를 구해보자.

1. $F(x)=x^2$

In [99]:
# 여기에 코드 작성
x = mg.Tensor(2.5)
F = x**2
F.backward()
x.grad

array(5.)

2. $F(x)=\cos{\sqrt{x}}$

In [104]:
x = mg.Tensor(2.5)
# 여기에 코드 작성
F = mg.cos(mg.sqrt(x))
F.backward()
x.grad

array(-0.31621085)

3. $F(x)=2+3x-5x^2$

In [102]:
# 여기에 코드 작성
F = -5*(x**2) + 3*x + 2
F.backward() 
x.grad

array(-22.)

4. $F(x)=e^{lnx}$

In [105]:
# 여기에 코드 작성
F = mg.e ** mg.log(x)
F.backward()
x.grad

array(1.)

5. $F(x)=(2xf(x))^2-f(x), f(x)=x^2$

In [106]:
# 여기에 코드 작성
f = x**2
F = (2*x*f)**2 - f
F.backward()
f.grad, x.grad

(array(311.5), array(2338.75))

In [5]:
import matplotlib.pyplot as plt
%matplotlib notebook

def plot_func_and_deriv(x, func):
    """
    함수 func(x)와 도함수 dfunc/dx를 같은 축(axis) 상에 그리는 함수
    
    매개변수 (Parameters)
    ----------
    x : mygrad.Tensor, shape-(N,)
        함수 func(x)와 도함수 dfunc/dx를 그리는 x의 정의역
    
    func: Callable[[Tensor], Tensor]
        x에 대한 일변수 함수
    
    반환 값 (Returns)
    -------
    Tuple[Figure, Axis]
        matplotlib로 그래프를 그리기 위한 fig와 ax
    """
    x = mg.tensor(x)
    y = func(x)
    y.backward()
    
    fig, ax = plt.subplots()
   
    # 여기에 코드 작성
    
    ax.plot(x.data, y.data, label="f(x)")
    ax.plot(x.data, x.grad, ls="--", label="df/dx")
    ax.grid(True)
    ax.legend()
    
    return fig, ax

2. 이제 위에서 작성한 함수를 이용하여 구간 $[0, 10]$를 균등하게 10,000개로 나눈 정의역에 대해 함수 $f(x) = \sin{(2x)}\; \cos{(x)}\; e^{-x/3}$와 그 도함수의 그래프를 그려보자.

In [6]:
def f(x): 
    # 여기에 코드 작성    
    return mg.sin(2*x)*mg.cos(x)*mg.exp(-x/3)

In [7]:
x = mg.linspace(0, 10, 10000)
fig, ax = plot_func_and_deriv(x, f)

<IPython.core.display.Javascript object>

ValueError: x, y, and format string must not be None

이러한 상황에 대해 아래의 코드를 채워 경사하강법을 수행하는 코드를 작성해보자. 제대로 코드를 작성한다면, 다음과 같이 $w=0$으로 서서히 가까워지는 결과가 출력될 것이다.
```
Tensor(4.)
Tensor(1.6)
Tensor(0.64)
Tensor(0.256)
Tensor(0.1024)
```

In [9]:
for _ in range(num_steps):
    ℒ = w ** 2
    ℒ.backward()
    # 여기에 코드 작성
    
    w -= learning_rate * w.grad
    
    print(w)

Tensor(4.)
Tensor(1.6)
Tensor(0.64)
Tensor(0.256)
Tensor(0.1024)


이러한 상황에 대해 아래의 코드를 채워 경사하강법을 수행하는 코드를 작성해보자. 제대로 코드를 작성한다면, 다음과 같이 $w_1=0, w_2=0$으로 서서히 가까워지는 결과가 출력될 것이다.
```
Tensor([1.2, 1.6])
Tensor([0.72, 0.64])
Tensor([0.432, 0.256])
Tensor([0.2592, 0.1024])
Tensor([0.15552, 0.04096])
```

In [12]:
const = np.array([2.0, 3.0])
for _ in range(num_steps):
    ℒ = const * w ** 2
    ℒ.backward()
    # 여기에 코드 작성
    
    w.data -= learning_rate * w.grad
    
    
    print(w)

Tensor([1.2, 1.6])
Tensor([0.72, 0.64])
Tensor([0.432, 0.256])
Tensor([0.2592, 0.1024])
Tensor([0.15552, 0.04096])


In [13]:
def gradient_step(tensors, learning_rate):
    """
    경사하강법의 공식에 따라 gradient-step을 실행.
    
    매개변수 (Parameters)
    ----------
    tensors : Union[Tensor, Iterable[Tensors]]
        단일 텐서, 혹은 텐서로 이루어진 iterable(리스트, 튜플 등) 모두 가능
        만약 특정 tensor에 대한 `tensor.grad`가 `None`인 경우, 업데이트를 건너 뜀

    learning_rate : float
        매 gradient-step에서의 학습률. 양수

    참고
    -----
    함수에서 진행되는 모든 gradient-steps는 tensor 내에서 바로 반영되므로, 반환 값 없음
    """
    # isinstance 함수를 이용하여 입력된 tensors가 단일 텐서인지, iterable인지 판단한다
    # 여기에 코드 작성
    if isinstance(tensors, mg.Tensor):
        
        tensors = [tensors]
    
    
    # for 문을 이용하여 tensors의 tensor를 하나씩 꺼내며 경사하강을 진행
    # 여기에 코드 작성
    
    for i in range(len(tensors)):
        tensors[i].data -= learning_rate * tensors[i].grad
    
    
    

앞서 수행했던 함수 $\mathscr{L}(w) = w^2$에 대한 경사하강법을 다시 한번 실행해봄으로써 보편적인 경사하강법 함수가 우리가 원하는대로 동작하는지 확인해보자.

In [14]:
w = mg.Tensor(10.0)
learning_rate = 0.3
num_steps = 5

In [15]:
for _ in range(num_steps):
    ℒ = w ** 2
    ℒ.backward()
    gradient_step(w, learning_rate=learning_rate)
    print(w)

Tensor(4.)
Tensor(1.6)
Tensor(0.64)
Tensor(0.256)
Tensor(0.1024)
